In [346]:
# Imports:

import numpy as np
import pandas as pd
import folium
from datetime import datetime

In [347]:
# Read the datasets that we will be using

df_005 = pd.read_csv("005.csv")
df_100 = pd.read_csv("100.csv")
df_1575 = pd.read_csv("1575.csv")
df_1640 = pd.read_csv("1640.csv")
df_1847 = pd.read_csv("1847.csv")
df_1848 = pd.read_csv("1848.csv")
df_1849 = pd.read_csv("1849.csv")
df_1850 = pd.read_csv("1850.csv")
df_1851 = pd.read_csv("1851.csv")

# Concatenate them into one big DataFrame
all_trucks_df = pd.concat([df_005, df_100, df_1575, df_1640, df_1847, df_1848, df_1849, df_1850, df_1851], ignore_index=True)

In general, the number of decimal places in latitude and longitude coordinates corresponds to a specific level of precision:

1 decimal place: Approximately 10 kilometers or 6.2 miles.

2 decimal places: Approximately 1 kilometer or 0.62 miles.

3 decimal places: Approximately 100 meters or 328 feet.

4 decimal places: Approximately 10 meters or 33 feet.

5 decimal places: Approximately 1 meter or 3.28 feet.

6 decimal places: Approximately 0.1 meter or 0.328 feet.

7 decimal places: Approximately 0.01 meter or 0.0328 feet.

We do not need measurements to the 100th of a meter. Measurements to 10 meters are more than sufficient for our needs. Rounding entries more specific than 10 meters to ~10 meters of accuracy will help us to group entries together and get more accurate ideas of where trucks are stopping at.

In [348]:
# Functions

# removes unimportant features from the data and removes any data points where the truck is moving
# returns a new dataframe with these features removed
def removeMovingData(dataFrame):
    new_df = dataFrame[(dataFrame['Type'] == 'Vehicle stopped') | (dataFrame['Type'] == 'Vehicle moving')]
    new_df.drop(columns=['Bearing (°)', 'Total Engine Hours', 'Location'], inplace=True)
    return new_df

# input: a dataframe with the format "Vehicle, Time, Elapsed Time Stopped, Latitude, Longitude" (from the function createTimeStoppedDF)
# output: a dataframe that contains a location, the number of times we stopped in that location, and the amount of time we spent in that location
# Adjust the constant ROUND to set how many decimal places to round the latitude/longitude to
def getTotalTimeStopped(dataFrame):
    ROUND = 3

    # Round the values of feature1 and feature2 to "ROUND" decimal places
    rounded_feature1 = dataFrame['Latitude'].round(ROUND)
    rounded_feature2 = dataFrame['Longitude'].round(ROUND)
    
    # Combine the rounded values to create a new Series with both features as index
    combined_series = rounded_feature1.astype(str) + ',' + rounded_feature2.astype(str)
    
    # Initialize an empty DataFrame to store combined information
    combined_info_df = pd.DataFrame({
        'Counts': combined_series.value_counts(),
        'Total Time Stopped': pd.to_timedelta('0 days')  # Initialize with zero time stopped
    })

    # Iterate over the combined_series and accumulate the time stopped for each unique combination of coordinates
    for combined_value, group in dataFrame.groupby(combined_series):
        total_time = group['Elapsed Time Stopped'].sum()
        combined_info_df.loc[combined_value, 'Total Time Stopped'] = total_time
    
    return combined_info_df

# input: given a dataframe of location history from motive
# output: returns a new dataframe with with the format "Vehicle, Time, Elapsed Time Stopped, Latitude, Longitude" This dataframe represents anytime a 
# truck stopped, and tracks where and for how long the truck was stopped
def createTimeStoppedDF(dataFrame):
    # Create an empty list to store dictionaries of rows
    rows_to_append = []

    previousStopped = None
    # Iterate through the original DataFrame
    for index, row in dataFrame.iterrows():
        if row['Type'] == 'Vehicle stopped':
            previousStopped = row
        elif row['Type'] == 'Vehicle moving' and previousStopped is not None and previousStopped['Vehicle'] == row['Vehicle']:
            timeStopped = datetime.strptime(row['Date/Time'], "%m/%d/%Y %I:%M %p") - datetime.strptime(previousStopped['Date/Time'], "%m/%d/%Y %I:%M %p")
            new_row = {
                'Vehicle': row['Vehicle'],
                'Time': previousStopped['Date/Time'],
                'Elapsed Time Stopped': timeStopped.total_seconds() / 60,
                'Latitude': previousStopped['Latitude'],
                'Longitude': previousStopped['Longitude']
            }
            # Append the dictionary to the list
            rows_to_append.append(new_row)

    # Create DataFrame from the list of dictionaries
    return pd.DataFrame(rows_to_append)

# convert minutes to the format to days:hours:minutes
def minutes_to_days_hours_minutes(minutes):
    # Calculate days, hours, and remaining minutes
    days = int(minutes // (24 * 60))
    hours = int((minutes % (24 * 60)) // 60)
    remaining_minutes = int(minutes % 60)
    
    # Format the result as string
    return f"{days}:{hours:02d}:{remaining_minutes:02d}"


In [349]:
# create a new dataframe that doesn't have all the unimportant data
importantCrumbs = removeMovingData(all_trucks_df)
importantCrumbs

C:\Users\mpbac\AppData\Local\Temp\ipykernel_20928\2741248215.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop(columns=['Bearing (°)', 'Total Engine Hours', 'Location'], inplace=True)


,Vehicle,Date/Time,Latitude,Longitude,Speed (mph),Odometer (mi),Total Fuel (gal),Type
6,5,4/1/2024 8:42 AM,40.755274,-112.011636,5.4,79914.77,196.66,Vehicle moving
9,5,4/1/2024 8:42 AM,40.755305,-112.012107,0.0,79914.79,196.67,Vehicle stopped
17,5,4/1/2024 8:49 AM,40.755305,-112.012107,6.3,79914.79,196.71,Vehicle moving
53,5,4/1/2024 8:55 AM,40.758462,-111.973770,0.0,79917.02,196.89,Vehicle stopped
63,5,4/1/2024 9:16 AM,40.758462,-111.973770,7.7,79917.32,196.97,Vehicle moving
...,...,...,...,...,...,...,...,...
293500,1851,5/1/2024 4:24 PM,40.755119,-112.012959,0.0,162187.42,29786.87,Vehicle stopped
293504,1851,5/1/2024 4:28 PM,40.755119,-112.012959,5.1,162187.42,29786.92,Vehicle moving
293512,1851,5/1/2024 4:29 PM,40.755085,-112.015762,0.0,162187.62,29786.97,Vehicle stopped
293519,1851,5/1/2024 4:36 PM,40.755085,-112.015762,5.4,162187.67,29787.07,Vehicle moving


In [350]:
# create a dataframe that contains all the stops
timeStopped_df = createTimeStoppedDF(importantCrumbs)
timeStopped_df

,Vehicle,Time,Elapsed Time Stopped,Latitude,Longitude
0,5,4/1/2024 8:42 AM,7.0,40.755305,-112.012107
1,5,4/1/2024 8:55 AM,21.0,40.758462,-111.973770
2,5,4/1/2024 9:21 AM,7.0,40.755193,-112.011737
3,5,4/1/2024 9:30 AM,40.0,40.755280,-112.011733
4,5,4/1/2024 10:14 AM,1.0,40.753848,-112.002407
...,...,...,...,...,...
7396,1851,5/1/2024 4:09 PM,2.0,40.760264,-111.973256
7397,1851,5/1/2024 4:14 PM,0.0,40.754690,-111.988556
7398,1851,5/1/2024 4:21 PM,1.0,40.759779,-112.012848
7399,1851,5/1/2024 4:24 PM,4.0,40.755119,-112.012959


In [351]:
# create a dataframe that contains the total time we spent at each location and how often we went there
combined_info_df = getTotalTimeStopped(timeStopped_df)
combined_info_df

C:\Users\mpbac\AppData\Local\Temp\ipykernel_20928\2741248215.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1187.0' has dtype incompatible with timedelta64[ns], please explicitly cast to a compatible dtype first.
  combined_info_df.loc[combined_value, 'Total Time Stopped'] = total_time


,Counts,Total Time Stopped
"40.755,-112.013",516,1177.0
"40.755,-112.012",413,50592.0
"40.756,-112.016",395,150804.0
"40.755,-112.016",361,25894.0
"40.759,-111.974",344,1283.0
...,...,...
"42.9,-112.464",1,0.0
"42.901,-112.464",1,0.0
"42.898,-112.463",1,1.0
"42.906,-112.466",1,1.0


In [352]:
# constants for the map
# adjust these to change what datapoints are displayed
TIMESSTOPPED = 6 # minimum number of times a location was stopped at
TOTALTIMESPENT = 60 # minimum amount of time trucks spent at a location

In [353]:
# Create a map that starts centered at a specific location
map_center = [40.75, -112]  # Center coordinates
mymap = folium.Map(location=map_center, zoom_start=12)  # Adjust zoom level as needed

# Add markers for each coordinate
for index, row in combined_info_df.iterrows():
    if row['Counts'] >= TIMESSTOPPED and row['Total Time Stopped'] >= TOTALTIMESPENT:
        coords = index.split(',')
        popup_text = f"Latitude: {coords[0]}, Longitude: {coords[1]}, Stops: {row['Counts']}, Total Time Stopped: {minutes_to_days_hours_minutes(row['Total Time Stopped'])}"
        folium.Marker([float(coords[0]), float(coords[1])], popup=popup_text).add_to(mymap)  # Convert coordinates to float

# Display the map
mymap
